<a href="https://colab.research.google.com/github/leeauk21/BA-model-Network/blob/master/VerySimpleMeanReversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
#your world trading api key
key='your world trading api key'

In [ ]:
class MeanReversion():
  def __init__(self,symbols: str,key: str,end: str):
    self.params = {'access_key': key,'limit':1000,'sort':'ASC','date_to':end,'symbols':symbols}
    api_result = requests.get('http://api.marketstack.com/v1/eod', self.params)
    api_response = api_result.json()
    self.df=pd.DataFrame(api_response['data'])
    self.df=self.df.set_index('date')
    #calculate 30MA
    self.df['MA30']=self.df['close'].rolling(30).mean()
    self.df['MA90']=self.df['close'].rolling(90).mean()
    # drop other cols except close price
    self.df=self.df[['close','MA30','MA90']]
    self.df.fillna(0,inplace=True)
  
  def output(self):
    return self.df
  

In [ ]:
# I am experiencing difficulties in merging this function with the class thus I am placing it outside the class
def check_signals(df):
  if df['MA30'] > df['MA90']:
    df['buy_signal']=int(1)
    df['sell_signal']=int(0)
  elif df['MA30'] < df['MA90']:
    df['sell_signal']=int(1)
    df['buy_signal']=int(0)
  else:
    df['sell_signal']=int(0)
    df['buy_signal']=int(0)
  return df

In [ ]:
def notify(df):
  df=df.reset_index()
  dictionary=df.to_dict('index')
  for i in dictionary.keys():
    day=dictionary[i]
    if day['buy_signal']==1.0:
      print(day['date']+'     '+'buy')
    elif day['sell_signal']==1.0:
      print(day['date']+'     '+'sell')

In [ ]:
#test
AAPL=MeanReversion('AAPL',key,'2020-09-03T00:00:00+0000')
AAPL_df=AAPL.output()
AAPL_df=AAPL_df.apply(check_signals,axis=1)
notify(AAPL_df)